In [1]:
# Imports
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.python.client import device_lib
import time
import seaborn as sns
import matplotlib.gridspec as gridspec

from keras.layers import Input, Dense
from keras import regularizers, Model
from keras.models import Sequential

from sklearn.manifold import TSNE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, precision_recall_curve, classification_report, confusion_matrix, average_precision_score, roc_curve, auc
%matplotlib inline

In [2]:
# Make sure that the dataset exists
for dirname, _, filenames in os.walk('/home/edinson/Documentos/U-distrital/Deteccion_total/nbaiot-dataset'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [3]:
def cargar_datos():
    data = pd.read_csv('/home/edinson/Documentos/U-distrital/nbaiot-dataset/5.benign.csv')
    columnas = data.columns
    df1 = pd.DataFrame(columns = columnas)
    df2 = pd.DataFrame(columns = columnas)
    
    devices_n = ['Danmini_Doorbell', 'Ecobee_Thermostat', 'Ennio_Doorbell', 'Philips_B120N10_Baby_Monitor', 'Provision_PT_737E_Security_Camera', 'Provision_PT_838_Security_Camera', 'Samsung_SNH_1011_N_Webcam', 'SimpleHome_XCS7_1002_WHT_Security_Camera', 'SimpleHome_XCS7_1003_WHT_Security_Camera']
    for device_id, device_name in enumerate(devices_n, 1):
        normal1 = pd.read_csv('/home/edinson/Documentos/U-distrital/nbaiot-dataset/{}.benign.csv'.format(device_id))
        df1 = df1.append(normal1, ignore_index=True, sort=False)
    
    n_X = df1.iloc[:,]
    n_X_scaled = MinMaxScaler().fit_transform(n_X.values)
    n_y = np.ones(n_X.shape[0]) # 1 represents normal

    devices_a = ['1.gafgyt.combo.csv', '1.gafgyt.junk.csv', '1.gafgyt.scan.csv', '1.gafgyt.tcp.csv', '1.gafgyt.udp.csv', '1.mirai.ack.csv', '1.mirai.scan.csv', '1.mirai.syn.csv', '1.mirai.udp.csv', '1.mirai.udpplain.csv']
    for device_id, device_name in enumerate(devices_a, 1):
        anomalo1 = pd.read_csv('/home/edinson/Documentos/U-distrital/nbaiot-dataset/{}'.format(device_name))
        df2 = df2.append(anomalo1, ignore_index=True, sort=False)
    
    a_X = df2.iloc[:,]
    a_X_scaled = MinMaxScaler().fit_transform(a_X.values)
    a_y = np.zeros(a_X.shape[0]) # 0 represents anomalous 

    return (n_X_scaled, n_y, a_X_scaled, a_y)
    

devices = ['Danmini_Doorbell', 'Ecobee_Thermostat', 'Ennio_Doorbell', 'Philips_B120N10_Baby_Monitor', 'Provision_PT_737E_Security_Camera', 'Provision_PT_838_Security_Camera', 'Samsung_SNH_1011_N_Webcam', 'SimpleHome_XCS7_1002_WHT_Security_Camera', 'SimpleHome_XCS7_1003_WHT_Security_Camera']
#devices = ['Danmini_Doorbell', 'Ecobee_Thermostat']
for device_id, device_name in enumerate(devices, 1):
    detect_zero_day_attack(device_name, device_id)
1.benign.csv
2.benign.csv
3.benign.csv
4.benign.csv
5.benign.csv
6.benign.csv
7.benign.csv
8.benign.csv
9.benign.csv
1.gafgyt.combo.csv
1.gafgyt.junk.csv
1.gafgyt.scan.csv
1.gafgyt.tcp.csv
1.gafgyt.udp.csv
1.mirai.ack.csv
1.mirai.scan.csv
1.mirai.syn.csv
1.mirai.udp.csv
1.mirai.udpplain.csv


import pandas as pd
devices = ['Danmini_Doorbell', 'Ecobee_Thermostat', 'Ennio_Doorbell', 'Philips_B120N10_Baby_Monitor', 'Provision_PT_737E_Security_Camera', 'Provision_PT_838_Security_Camera', 'Samsung_SNH_1011_N_Webcam', 'SimpleHome_XCS7_1002_WHT_Security_Camera', 'SimpleHome_XCS7_1003_WHT_Security_Camera']
#devices = ['Danmini_Doorbell', 'Ecobee_Thermostat']
anomalos = ['1.gafgyt.combo.csv', '1.gafgyt.junk.csv', '1.gafgyt.scan.csv', '1.gafgyt.tcp.csv', '1.gafgyt.udp.csv', '1.mirai.ack.csv', '1.mirai.scan.csv', '1.mirai.syn.csv', '1.mirai.udp.csv', '1.mirai.udpplain.csv']
for device_id, device_name in enumerate(devices, 1):
    detect_zero_day_attack(device_name, device_id)

In [4]:
# AutoEncoder

def generate_and_train_autoencoder(X):
    ## input layer 
    input_layer = Input(shape=(X.shape[1],))

    ## encoding part
    encoded = Dense(100, activation='tanh', activity_regularizer=regularizers.l1(10e-5))(input_layer)
    encoded = Dense(50, activation='relu')(encoded)

    ## decoding part
    decoded = Dense(50, activation='tanh')(encoded)
    decoded = Dense(100, activation='tanh')(decoded)

    ## output layer
    output_layer = Dense(X.shape[1], activation='relu')(decoded)

    autoencoder = Model(input_layer, output_layer)
    autoencoder.compile(optimizer="adadelta", loss="mse")

    autoencoder.fit(X[0:1000], X[0:1000], 
                    batch_size = 256, epochs = 10, 
                    shuffle = True, validation_split = 0.20);

    return autoencoder

In [5]:
# hidden representation
def get_hidden_representation_normal_anomalous(autoencoder, n_X, a_X):
    hidden_representation = Sequential()
    hidden_representation.add(autoencoder.layers[0])
    hidden_representation.add(autoencoder.layers[1])
    hidden_representation.add(autoencoder.layers[2])

    normal_hid_rep = hidden_representation.predict(n_X[:3000])
    anomalous_hid_rep = hidden_representation.predict(a_X[:3000])

    return (normal_hid_rep, anomalous_hid_rep)

In [6]:
def tsne_plot(x, y, title="Scatter Plot", name="graph.png"):
    tsne = TSNE(n_components=2, random_state=0)
    X_t = tsne.fit_transform(x)

    plt.figure(figsize=(12, 8))
    plt.scatter(X_t[np.where(y == 1), 0], X_t[np.where(y == 1), 1], marker='o', color='g', linewidth='1', alpha=0.8, label='Normal')
    plt.scatter(X_t[np.where(y == 0), 0], X_t[np.where(y == 0), 1], marker='o', color='r', linewidth='1', alpha=0.8, label='Anomalous')

    plt.title(title)
    plt.legend(loc='best');
    plt.savefig('{}-{}'.format(title,name));
    plt.show();

In [7]:
# Combine normal and anomalous data
def combine_normal_anomalous(normal, anomalous):
    X = np.append(normal, anomalous, axis = 0)
    y_n = np.ones(normal.shape[0])
    y_a = np.zeros(anomalous.shape[0])
    y = np.append(y_n, y_a)

    return (X, y)

In [8]:
# Evaluate model's performace
def evaluate_model(X, y):
    train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2)
    clf = LogisticRegression(solver="lbfgs").fit(train_X, train_y)
    pred_y = clf.predict(test_X)
    return (test_y, pred_y)

In [9]:
# Show model's report
def show_model_report(title, X, y, hid_X, hid_y, test_y, pred_y):
    #tsne_plot(X, y, '{}-original'.format(title), "original.png")
    #tsne_plot(hid_X, hid_y, title, "hidden_representation.png")

    print(title)
    print ("")
    print ("Classification Report: ")
    print (classification_report(test_y, pred_y))
    
    print ("")
    print("MATRIX DE CONFUSIÓN, PREDICCIONES CON LOS VALORES ESPERADOS ")
    print(confusion_matrix(test_y, pred_y))

    print ("")
    print ("Accuracy Score: ", accuracy_score(test_y, pred_y))

In [10]:
def detect_zero_day_attack(device_name):
    (n_X_scaled, n_y, a_X_scaled, a_y) = cargar_datos()
    autoencoder = generate_and_train_autoencoder(n_X_scaled)
    (normal_hid_rep, anomalous_hid_rep) = get_hidden_representation_normal_anomalous(autoencoder, n_X_scaled, a_X_scaled)
    (X, y) = combine_normal_anomalous(n_X_scaled, a_X_scaled)
    (hid_X, hid_y) = combine_normal_anomalous(normal_hid_rep, anomalous_hid_rep)
    (test_y, pred_y) = evaluate_model(hid_X, hid_y)
    show_model_report(device_name, X, y, hid_X, hid_y, test_y, pred_y)

In [11]:
devices = ['detección_total']
#devices = ['Danmini_Doorbell', 'Ecobee_Thermostat']
for device_id, device_name in enumerate(devices, 1):
    detect_zero_day_attack(device_name)

Epoch 1/10
4/4 [==============================] - 1s 151ms/step - loss: 0.0369 - val_loss: 0.0355
Epoch 2/10
4/4 [==============================] - 0s 43ms/step - loss: 0.0369 - val_loss: 0.0355
Epoch 3/10
4/4 [==============================] - 0s 53ms/step - loss: 0.0369 - val_loss: 0.0355
Epoch 4/10
4/4 [==============================] - 0s 49ms/step - loss: 0.0369 - val_loss: 0.0355
Epoch 5/10
4/4 [==============================] - 0s 35ms/step - loss: 0.0369 - val_loss: 0.0354
Epoch 6/10
4/4 [==============================] - 0s 38ms/step - loss: 0.0368 - val_loss: 0.0354
Epoch 7/10
4/4 [==============================] - 0s 48ms/step - loss: 0.0368 - val_loss: 0.0354
Epoch 8/10
4/4 [==============================] - 0s 42ms/step - loss: 0.0368 - val_loss: 0.0354
Epoch 9/10
4/4 [==============================] - 0s 36ms/step - loss: 0.0368 - val_loss: 0.0354
Epoch 10/10
4/4 [==============================] - 0s 36ms/step - loss: 0.0368 - val_loss: 0.0354
detección_total

Classificat